In [128]:
import pandas as pd
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
import pymorphy2
from operator import itemgetter
nltk.download('vader_lexicon')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\12\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\12\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\12\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [129]:
# get: series of text to preprocess
# return series of preprocessed text
def preprocess(s):
    vader = SentimentIntensityAnalyzer()
    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english'))
    ret = pd.Series(index=s.index)
    for i in s.index:
        reply = s[i]
        # remove account including
        reply = re.sub(r'@[\w]+', ' ', reply)
        # remove links
        reply = re.sub(r'https?:[\w./]*', ' ', reply)
        reply = reply.lower()
        # remove all non-words
        reply = re.sub(r'[^a-z]', ' ', reply)        
        # lemmatization
        prepared = [lemmatizer.lemmatize(word) for word in reply.split()
                    if lemmatizer.lemmatize(word) not in stop_words]
        ret[i] = ' '.join(prepared)
    return ret


#get: dataframe of preprocessed text
#return dataframe with two new columns: class and score
def getClass(df):
    #ret = pd.Series(index=s.index)
    for i in df.index:
        text = df['preprocessed'][i]
        res = vader.polarity_scores(text)
        # there is useless in our problem
        del res['compound']
        # get key (pos, neu or neg) with max value
        df.loc[i, 'class'] = max(res.items(), key=itemgetter(1))[0]
        df.loc[i, 'score'] = max(res.values())
    return df
        
df = pd.read_excel('data.xlsx')
df = df.drop(df.columns[0:8], axis=1)
df['preprocessed'] = preprocess(df['text'])
df = getClass(df)
df.to_excel('result.xlsx')
df[df['score'] != 0].head(15)

,text,preprocessed,class,score
0,@Microsoft All d more reasons why @Microsoft i...,reason better place work,neu,0.508
1,@Microsoft @NCCEducation Great idea,great idea,pos,0.804
3,@Microsoft Wow! That's nice!,wow nice,pos,1.000
4,@Microsoft Microsoft - If not already any chan...,microsoft already chance could make office ava...,neu,0.833
6,@Microsoft 👏 people over profit,people profit,pos,0.744
7,@Microsoft Guilt trip,guilt trip,neg,0.677
8,@Microsoft Well Done to you Microsoft for this...,well done microsoft initiative please life ret...,neu,0.440
9,@Microsoft In this really hard situation you s...,really hard situation make office program free...,neu,0.546
10,@Microsoft Something seriously wrong with @Mic...,something seriously wrong tier two support thr...,neu,0.483
11,@Microsoft Then why aren't vendors who are qua...,vendor quarantined sick getting paid seems sil...,neu,0.577


# Описание и другие возможности
Для получения классов я использовал VADER, т.к. эта библиотека заточена для обработки текстов из социальных сетей. Какие ещё есть варианты?
* TextBlob. Пожалуй, самая простая вещь, встроена в nltk. Но по качеству чуть хуже VADER'а
* FastText - отличная библиотека для генерации word2vec. Однако требует обучения, т.е. ручной постановки меток на текстах. В силу малого количества времени не подходит.
Если бы времени было больше, я бы попробовал обучить сеточку на уже маркированных открытых наборах текстов.
# Workflow
В принципе, он виден по функциям выше. Коротко перечислим основные моменты:
* Скачивание данных. К сожалению, API Твиттера доступно только после регистрации приложения. Я оставил заявку, но они рассматриваются в ручном режиме в течение 2-3 дней. Поэтому я использовал сторонние решения для скачивания ответов на твиты, однако эти решения имеют сильные ограничения. В принципе, т.к. я использовал готовую модель, а не обучал что-то, большое количество данных и не нужно.
* Предобработка. Удаление ников, ссылок, перевод в нижний регистр
* Лемматизация. С английским языком это несложно
* Удаление стоп-слов: они нейтральны по смыслу, но добавлюят шума.
* Получения класса
